# 基于知识图谱的RAG

传统RAG框架在处理复杂查询时仍存在明显局限。

依赖非结构化文本向量检索的方式，往往难以捕捉实体间的深层关系，导致上下文检索不精确、信息碎片化，甚至诱发模型产生"幻觉"。

通过利用知识图谱的显式语义关系和图结构优势，GraphRAG能够提供更精准的上下文检索和更强的推理能力，在多跳查询和事实性要求较高的场景中表现尤为出色。

## 从传统RAG到知识图谱增强RAG的演进

![](images/7_1_1.svg)


### 传统RAG框架的固有局限性

- 关系理解的缺失：

    - 基于向量的检索**主要关注语义相似性**，难以捕捉和利用实体之间复杂的、隐含的关系。
    - 当查询涉及多实体关联或因果推理时，检索到的文本块之间可能缺乏逻辑联系，导致LLM难以进行有效的综合与推理。

- 上下文的碎片化：

    - 文本被切分成独立的块进行索引，这破坏了原文的结构和上下文连续性。
    - 对于需要跨越多个文档或长距离文本进行信息整合的问题，传统RAG往往力不从心。

- 检索噪声与幻觉风险：

    - 检索过程可能返回不相关或仅部分相关的噪声信息，这会干扰LLM的判断，甚至诱发其产生与事实不符的"幻觉"内容。

- 推理能力有限：
  
    - 传统 RAG 的推理能力受限于检索到的线性文本内容，**难以支持**需要结构化知识进行导航的多跳推理。

- 跨文档联结能力弱：

    - 即便采用较大的切块与滑窗重叠，跨文档/跨章节的实体共现与隐式引用仍难以被显式建模，导致需要“连边”才能回答的问题召回不足。

- 实体歧义与别名问题：

    - **仅依赖向量相似度**很难稳定地区分同名实体（如不同城市/公司/人名），或统一别名、缩写与多语言变体，易造成错误对齐与语义漂移。

- 时效性与版本一致性不足：

    - 文本块往往缺少可计算的时间属性与有效期边界，导致时序敏感问题（如“某公司在某年是否仍为子公司”）难以得到一致答案。

### 知识图谱 RAG 的优势

知识图谱通过节点（实体）和边（关系）的网络结构，将离散的知识显式地组织成一个相互连接的语义网络，可以克服传统RAG的局限性。

- 结构化语义表达：

    - 知识图谱以图的形式直接编码实体间的显式语义关系（如"公司A-收购-公司B"），
    - 避免了LLM从文本中进行隐式、可能存在偏差的推断，为复杂查询提供了清晰的导航路径。

- 增强推理能力：

    - 图的结构天然支持多跳推理，系统可以沿着图中的路径遍历，发现间接但关键的知识关联，从而回答需要多步逻辑推导的复杂问题。

- 事实性与可解释性：

    - 基于知识图谱检索的答案可以追溯其在图中的推理路径，这为答案提供了事实依据，极大地增强了系统的可解释性和可信度，有效抑制了幻觉。

- 异构数据集成：

    - 知识图谱能够无缝集成来自不同来源的结构化（如数据库）和非结构化（如文本）数据，
    - 形成一个全面、统一的知识视图，这在金融、医疗等需要整合海量异构信息的领域尤为重要。

- 语义模式与本体支撑：

    - 通过模式（Schema）与本体（Ontology）明确实体类型、关系约束与属性域值，约束推理空间，降低歧义。

- 溯源与置信度管理：

    - 边与节点可携带来源、时间戳与置信度，便于在生成时进行来源归因与冲突消解。

- 时间与版本建模：

    - 支持时间态边与版本化节点，允许对“历史状态”与“有效区间”进行查询（Time-travel Query）。


GraphRAG将检索的目标从独立的文本片段转变为在知识图谱中寻找相关的实体、关系、路径或子图。

在上下文召回、多跳问答准确率、事实一致性与可解释性等关键指标上较传统RAG具有显著优势。

这标志着RAG技术从“信息检索”向“知识利用”的范式演进。

## GraphRAG框架的核心架构与工作流程

大多数GraphRAG框架遵循一个通用的三阶段流程：知识图谱构建、图谱检索与增强生成。

### 通用架构三阶段


#### 知识图谱构建

该阶段的目标是从原始数据中构建一个高质量的知识图谱。

- 知识抽取：利用 LLM 或 IE 管线从非/半结构化文本中抽取实体、关系与属性，包含指代消解、别名归一（Normalization）与术语标准化。

- 质量控制：对三元组进行置信度评估、人机协同抽检与冲突消解；必要时通过规则与本体约束做一致性校验。

- 图谱融合：进行实体对齐与去重（Entity Resolution），合并跨来源知识并保留来源/时间戳等溯源信息。

- 存储与索引：落地到图数据库（如Neo4j/NebulaGraph/TigerGraph/Neptune等）并建立必要的属性与关系索引，便于后续高效检索与遍历。



#### 图谱检索

当用户提出查询时，系统不再是简单地进行向量相似度搜索，而是执行更复杂的图谱检索操作。

1. 实体定位：先用实体链接（Entity Linking）或向量检索在图中锁定核心实体节点。

2. 子图探索：从命中节点出发，利用图查询语言（如Cypher）或遍历算法进行邻域扩展、路径发现与约束过滤（例如限定关系类型、跳数、时间区间与置信度阈值）。

3. 结构化证据抽取：将路径与关键节点属性序列化为可读证据片段，或生成子图的文本摘要。

4. 高级检索：如GraphRAG采用社区检测（如Leiden）生成多层次摘要，实现“全局-局部”联合检索。

#### 增强生成

将检索到的结构化知识（如相关实体的属性、实体间的关系路径、描述子图的文本摘要等）与原始查询一同注入到 LLM 提示（Prompt）中。

- 提示设计：明确要求模型引用“图证据”，并在回答中给出来源与路径（可作为可选的引用附录）。

- 证据融合：将“结构化三元组/路径”与“原文片段”联合提供，兼顾事实性与文本细节。

- 约束回答：对敏感领域（财务/医疗）可采用模板化输出与字段校验，减少幻觉。

### 方法论分类

根据知识图谱与 RAG 结合深度的不同，GraphRAG 方法大致可以归为三类。

- 知识驱动型： 检索过程主要或完全依赖于知识图谱。直接在图上进行查询与推理（如文本转 Cypher/Gremlin），适用于需要强逻辑约束与可解释性的任务。

- 索引驱动型： 将知识图谱的结构信息融入到文本索引。例如用邻居实体/关系充当元数据特征、或将子图摘要拼接到文本后再向量化，以提升召回与重排效果。

- 混合型： 同时使用图检索与文本检索，并对结果统一重排与融合。

    - 简单事实问答可优先图检索；叙事性/描述性问题可补充文本证据。
    - 对复杂查询，可先图上定位路径，再以路径节点为“导航枢纽”做针对性文档检索。

**优劣对比**

- 知识驱动：高精度/高可解释，但覆盖与容错受限于图谱完备性。

- 索引驱动：集成成本低，但显式推理弱、可解释性一般。

- 混合型：综合表现更稳健，但系统复杂度与工程成本较高。

## 前沿GraphRAG框架介绍

### GraphRAG (Microsoft)

GraphRAG是一个重量级的框架，其核心思想是“先构建全局知识，再按需检索。

1. 文本→三元组/子图构建→图谱（属性图）；

2. 采用社区检测（如Leiden）进行图划分，按层级生成全局/社区/局部摘要；

3. 查询时先匹配全局/社区摘要，再下钻到局部子图与原文证据；

4. 根据需要返回“全局观”与“证据路径”并行的双视角答案。

优势是提供强全局感知与良好可解释性，适合探索性分析、全景总结与多跳问答。

但前期构建与分层摘要成本较高，对数据持续变更的场景需设计批处理/增量更新与缓存策略。

### FRAG (Flexible RAG)

强调“灵活性”与“模块化”，以自适应不同复杂度的查询。

- 查询分流：通过分类器或规则将请求判定为简单/复杂。

    - 简单查询：直接实体链接+属性查找，低延迟返回。
    
    - 复杂查询：激活路径检索/多跳推理模块，再融合文本证据。

- 易扩展：模块化设计与预训练LLM能力复用，迁移成本低。

### GraphIRAG (Iterative Knowledge Retrieval)

引入“迭代检索”思想。

- 由控制器在生成中多轮触发图查询，逐步补齐证据链。

- 以“新信息增益”为准则决定是否继续迭代与何时停止。

- 对时间敏感与多跳推理任务更具鲁棒性。

## 性能评估与基准测试

### 评估指标

主要涵盖三个方面：检索质量、生成质量、系统性能。

**检索质量：**

- 上下文精确率（Context Precision）：检索到的上下文中“真正相关”的比例；

- 上下文召回率（Context Recall）：所有应当支持答案的“金标准证据”被检回的比例；

- 引用/归因准确率（Citation/Attribution）：生成答案中的关键断言是否被检索证据正确支撑。


**生成质量：**

- 问答：精确匹配（EM）、F1；

- 摘要：ROUGE；

- 事实一致性/忠实度（Faithfulness）：断言与证据的一致性，可配合“逐句打标+归因检查”。

**系统性能：**

- 推理延迟（Latency）：从接收查询到返回最终答案所需的总时间，是衡量系统响应速度的核心指标。

- 吞吐量（Throughput）：系统在单位时间内能处理的查询数量（QPS）。

- 成本（Cost）：包括API调用次数、计算资源（GPU/CPU）和内存消耗等。


### 常用基准数据集

多跳问答数据集：HotpotQA、2WikiMultihopQA 和 MuSiQue。

- 要求模型在多个知识源之间进行推理，是检验GraphRAG核心能力的试金石。

复杂问答数据集：WebQSP 和 ComplexWebQuestions (CWQ)。

- 包含结构复杂的查询。

知识图谱驱动的QA数据集：KGQAgen-10k。

- 专门为评估知识图谱问答能力设计。

现有基准（如WebQSP）存在数据质量问题或评估方法过于僵化（如严格的EM），可能导致对模型性能的低估。

因此，构建更高质量、更科学的基准是未来的一个重要方向。

## 生产环境部署实践与挑战

- 知识图谱的构建与动态维护：

  - 高质量知识图谱的构建本身就是一项耗时耗力的知识工程。
  - 在生产环境中，知识需要持续保持最新状态，如何设计高效、准确、低成本的动态更新机制是核心难点。

- 系统性能与可扩展性：

    - 随着知识库规模和用户查询量的增长，系统的响应延迟、吞吐量和资源消耗成为主要瓶颈。
    - 生产部署需要应对向量化速度慢、内存溢出、API 失败等具体问题。

- 安全与隐私保护：

    - RAG 系统引入外部数据源，带来了新的安全风险，包括数据隐私泄露、模型被注入恶意数据（模型中毒）、以及针对检索模块的攻击等。

- 成本控制：

    - 大规模部署 GraphRAG 系统需要大量的计算和存储资源，如何优化系统架构以降低运营成本是一个重要的商业考量。